# DAY3 課題
山口 祐治

## 先週の課題の振り返り
- 無条件に外した変数についての再評価
- yr_renovatedの変数変換

## DAY3 課題
- 識別モデルに使えそうなアルゴリズムを決める(2つ以上)。
- 質的変数が扱えないアルゴリズムを使う場合は、ダミー変数に置き換える。

In [190]:
%matplotlib inline
from IPython.core.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #3D散布図の描画
import itertools #組み合わせを求めるときに使う
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pyper #pyperとRを事前にインストールしておく必要がある

## データの読み込み

In [191]:
df_data = pd.read_csv("../1_data/kc_house_data.csv")
df_data["price"] = df_data["price"] / 10**6 #単位を100万ドルにしておく
df_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,0.2219,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,0.5380,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,0.1800,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,0.6040,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,0.5100,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 無条件に外した変数についての再評価
先週は、まず、無条件にdate, zipcode, lat, longの値が不要そうだと考えて外していた。この想定のもとでVIFを計算し、

- sqft_living
- sqft_above
- sqft_basement

に多重共線性があるとの結論に達し、sqft_aboveとsqft_basementを外す方針とした。

この段階でいったん予測性能を決定係数の指標で評価してみる。

In [197]:
# 目的変数
y_var = "price"
# 説明変数(DAY2で方針を決定した12変数のみとする)
X_var = ["bedrooms", "bathrooms","sqft_living","floors","waterfront","view","condition","grade",
         "yr_built","yr_renovated","sqft_living15","sqft_lot15"]

X = df_data[X_var]
y = df_data[y_var]

# データを訓練データ80%、テストデータ20%に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = LinearRegression(fit_intercept=True)
regr.fit(X_train, y_train)

#テストデータの決定係数の表示
print("test score: ", regr.score(X_test, y_test))

test score:  0.650361135601811


あまりよい値ではないような気がする。そもそもすべてのパラメータを使ってみたらどうだろう。
ただし、dateについては文字列型で変換が必要なのと、値としては2014年から2015年のデータでほぼ差異がないことからそれほど解析データの意味はないと考え、いったん考えないこととする。

In [199]:
X = df_data.drop(["id", "date", "price", "sqft_above", "sqft_basement"], axis=1);
y = df_data["price"]

# データを訓練データ80%、テストデータ20%に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = LinearRegression(fit_intercept=True)
regr.fit(X_train, y_train)

#テストデータの決定係数の表示
print("test score: ", regr.score(X_test, y_test))

test score:  0.7089225842672379


zipcode, lat, longのデータを追加したところ、決定係数の値に有意な差がみられた。地域による差があるものと思われる

※本来は交差検証などの手法を使って評価すべきと思われるが、今は何回か実施してみて値のばらつきを確認する程度にとどめている。

続いて、sqft_livingを使う代わりに、sqft_aboveとsqft_basementを使ったらどうなるか検証してみる。

In [194]:
#sqft_livingを除いてやってみる
X = df_data.drop(["id", "date", "price", "sqft_living"], axis=1);
y = df_data["price"]

# データを訓練データ80%、テストデータ20%に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = LinearRegression(fit_intercept=True)
regr.fit(X_train, y_train)

#print("傾き=%s"%regr.coef_,"切片=%s"%regr.intercept_,"パラメータ=%s"%regr.get_params())

#テストデータの決定係数の表示
print("test score: ", regr.score(X_test, y_test))

test score:  0.702802098853257


それほど違いがないように見える。結論としては、 zipcode, lat, longの値を無条件に外したのが間違いであることが分かった。

## yr_renovatedの変数変換

前回、
「yr_renovatedについては、リフォームした場合は西暦、リフォームしていない場合はゼロが入っているようなので、データとしてあまりよくないように思える。」
といったコメントを残したが、丸尾先生がslackでyr_renovatedを0,1の値にしてはどうかというコメントがあり、試してみることにした。

In [195]:
df_data2 = pd.read_csv("../1_data/kc_house_data.csv")
df_data2["price"] = df_data2["price"] / 10**6 #単位を100万ドルにしておく

# yr_renovatedを0 or 1に変換してみる
display(df_data2.head())
df_data2["yr_renovated"] = df_data2["yr_renovated"].apply( lambda x: 1 if x > 0 else 0 )
display(df_data2.head())

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,0.2219,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,0.5380,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,0.1800,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,0.6040,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,0.5100,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,0.2219,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,0.5380,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,0.1800,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,0.6040,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,0.5100,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [200]:
X = df_data2.drop(["id", "date", "price", "sqft_above", "sqft_basement"], axis=1);
y = df_data2["price"]

# データを訓練データ80%、テストデータ20%に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = LinearRegression(fit_intercept=True)
regr.fit(X_train, y_train)

#テストデータの決定係数の表示
print("test score: ", regr.score(X_test, y_test))

test score:  0.7095063315902557


それほど違いがないように見える。yr_renovatedの数値変換はそれほど意味がなさそう

## 識別モデルに使えそうなアルゴリズムを決める(2つ以上)。
決定着とランダムフォレストを試してみました。計算方法、評価方法含めてこれでよいか分かっていません（笑）。

In [247]:
#
# 決定木
#
X = df_data2.drop(["id", "date", "price", "sqft_above", "sqft_basement"], axis=1);
y = df_data2["price"]

# データを訓練データ80%、テストデータ20%に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor()
regr = regr.fit(X_train, y_train)

#テストデータの決定係数の表示
print("test score: ", regr.score(X_test, y_test))

test score:  0.735207336307709


In [252]:
#
# ランダムフォレスト
#
X = df_data2.drop(["id", "date", "price", "sqft_above", "sqft_basement"], axis=1);
y = df_data2["price"]

# データを訓練データ80%、テストデータ20%に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr = clf.fit(X_train, y_train)

#テストデータの決定係数の表示
print("test score: ", regr.score(X_test, y_test))

test score:  0.7843811437962842


## 質的変数が扱えないアルゴリズムを使う場合は、ダミー変数に置き換える。
今回の家のデータの場合に該当するものがちょっとよく分かりませんでした。ご教授いただければ幸いです。